In [1]:
import camelot
import pandas as pd
import json
import os
import re
import sys


d:\Users\juanp_schamun\AppData\Local\anaconda3\envs\pdf_env\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
import camelot
import pandas as pd
import json
import os
import re
import sys

def extract_tables_from_pdf(pdf_path, output_json_path):
    """
    Extrae tablas de un archivo PDF y las guarda en un archivo JSON.
    Este script está optimizado para la estructura del PDF de GENTECH y
    ahora incluye una lógica para detectar y categorizar los productos,
    manejando celdas combinadas de forma más robusta.

    Args:
        pdf_path (str): La ruta del archivo PDF de entrada.
        output_json_path (str): La ruta donde se guardará el archivo JSON de salida.
    """
    # Verificar si el archivo PDF existe
    if not os.path.exists(pdf_path):
        print(f"Error: El archivo PDF '{pdf_path}' no se encontró.")
        return

    try:
        # 1. Extracción con coordenadas específicas
        print(f"🕵️‍♂️ Extrayendo tablas de '{pdf_path}' usando coordenadas y el método 'stream'...")
        tables = camelot.read_pdf(pdf_path, flavor='stream', pages='1', table_areas=['50,780,800,0'])

        if tables.n == 0:
            print("❌ No se encontraron tablas. Verifique las coordenadas o el tipo de tabla.")
            return

        all_tables_data = []

        # 2. Transformación y Limpieza de la tabla extraída
        for i, table in enumerate(tables):
            df = table.df.copy()
            
            # Limpiar los datos de la tabla, eliminando espacios extra
            df = df.map(lambda x: str(x).strip() if isinstance(x, str) else x)

            # Eliminar filas completamente vacías
            df.dropna(how='all', inplace=True)
            
            # Eliminar la fila de encabezado que a veces se extrae
            df = df[~df.iloc[:, 0].str.contains('DESCRIPCIÓN', na=False)]
            
            # Reorganizar los encabezados y datos
            df.columns = ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']

            products = []
            current_category = 'Sin Categoria'
            last_product_desc = ""

            for idx, row in df.iterrows():
                row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
                
                # Detectar las filas de categorías
                category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                                      'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
                is_category_row = any(keyword in row_values_str for keyword in category_keywords)

                if is_category_row:
                    category_name = next((val for val in row.tolist() if str(val).strip()), None)
                    if category_name:
                        cleaned_name = re.sub(r'(LÍNEA|LINEA)\s*', '', str(category_name), flags=re.IGNORECASE).strip()
                        if cleaned_name.upper() not in ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']:
                            current_category = cleaned_name
                    continue

                # Si la fila tiene una descripción no vacía, es un producto principal
                if str(row['DESCRIPCIÓN']).strip():
                    last_product_desc = str(row['DESCRIPCIÓN'])

                # Comprobar si la fila contiene múltiples entradas de producto debido a celdas combinadas
                # y dividir todas las columnas relevantes por el salto de línea
                if '\n' in str(row['PRESENTACIÓN']) or '\n' in str(row['CONTENIDO NETO']) or '\n' in str(row['PRECIO UNITARIO CON IVA']):
                    presentaciones = str(row['PRESENTACIÓN']).split('\n')
                    contenidos = str(row['CONTENIDO NETO']).split('\n')
                    cantidades = str(row['CANTIDAD POR BULTO']).split('\n')
                    precios_netos = str(row['PRECIO UNITARIO NETO']).split('\n')
                    precios_iva = str(row['PRECIO UNITARIO CON IVA']).split('\n')

                    # Iterar sobre las partes divididas para crear un producto por cada una
                    # Ahora se usa el largo de la lista de precios_iva, que es el más fiable
                    for j in range(len(precios_iva)):
                        product_row = {
                            'Categoria': current_category,
                            'DESCRIPCIÓN': last_product_desc,  # Asignamos la última descripción válida
                            'PRESENTACIÓN': presentaciones[j].strip() if j < len(presentaciones) else '',
                            'CONTENIDO NETO': contenidos[j].strip() if j < len(contenidos) else '',
                            'CANTIDAD POR BULTO': cantidades[j].strip() if j < len(cantidades) else '',
                            'PRECIO UNITARIO NETO': precios_netos[j].strip() if j < len(precios_netos) else '',
                            'PRECIO UNITARIO CON IVA': precios_iva[j].strip()
                        }
                        products.append(product_row)
                
                # Si es una fila de producto normal (sin celdas combinadas), lo procesamos
                elif str(row['PRECIO UNITARIO CON IVA']).strip():
                    product_row = {
                        'Categoria': current_category,
                        'DESCRIPCIÓN': last_product_desc,
                        'PRESENTACIÓN': row['PRESENTACIÓN'],
                        'CONTENIDO NETO': row['CONTENIDO NETO'],
                        'CANTIDAD POR BULTO': row['CANTIDAD POR BULTO'],
                        'PRECIO UNITARIO NETO': row['PRECIO UNITARIO NETO'],
                        'PRECIO UNITARIO CON IVA': row['PRECIO UNITARIO CON IVA']
                    }
                    products.append(product_row)
            
            all_tables_data.append({"tabla_1": products})

            return products

        # # 3. Carga: Escribir los datos en el archivo JSON
        # with open(output_json_path, 'w', encoding='utf-8') as f:
        #     json.dump(all_tables_data, f, ensure_ascii=False, indent=4)
        
        # print(f"✅ ¡Éxito! {tables.n} tablas extraídas y guardadas en '{output_json_path}'.")

    except Exception as e:
        print(f"❌ Ocurrió un error inesperado: {e}")
        # En caso de error, imprimir el DataFrame para depuración
        try:
            print("--- Contenido del DataFrame para depuración ---")
            print(df)
            print("---------------------------------------------")
        except NameError:
            pass


In [3]:
pdf_file_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
output_json_file = 'test.json'

tablas = extract_tables_from_pdf(pdf_file_path, output_json_file)
tablas

🕵️‍♂️ Extrayendo tablas de 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf' usando coordenadas y el método 'stream'...


[{'Categoria': 'Sin Categoria',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO NETO': '',
  'PRECIO UNITARIO CON IVA': 'IVA'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': 'POTE -150 COMPRIMIDOS',
  'CONTENIDO NETO': '285',
  'CANTIDAD POR BULTO': '12',
  'PRECIO UNITARIO NETO': '$',
  'PRECIO UNITARIO CON IVA': '$'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO NETO': '11.424,79',
  'PRECIO UNITARIO CON IVA': '13.824,00'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': 'POTE - 325 COMPRIMIDOS',
  'CONTENIDO NETO': '618',
  'CANTIDAD POR BULTO': '12',
  'PRECIO UNITARIO NETO': '$',
  'PRECIO UNITARIO CON IVA': '$'},
 {'Categoria': 'ALTO RENDIMIENTO',
  'DESCRIPCIÓN': '',
  'PRESENTACIÓN': '',
  'CONTENIDO NETO': '',
  'CANTIDAD POR BULTO': '',
  'PRECIO UNITARIO 

In [30]:
pdf_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
tables = camelot.read_pdf(pdf_path, flavor='hybrid', pages='1', table_areas=['50,780,800,0'])


d:\Users\juanp_schamun\AppData\Local\anaconda3\envs\pdf_env\lib\site-packages\camelot\parsers\hybrid.py:161: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table.df = table.df.replace("", np.nan)


In [31]:
tables[0].df.head(20)

,0,2,3,4,5,6
0,,,NETO GRAMOS,POR BULTO,UNITARIO NETO,
1,,,,,,IVA
2,,LINEA ALTO RENDIMIENTO,,,,
3,,POTE -150 COMPRIMIDOS,285,12,"$ \n11.424,79","$ \n13.824,00"
4,AMINO 7600,,,,,
5,,POTE - 325 COMPRIMIDOS,618,12,"$ \n22.849,59","$ \n27.648,00"
6,AMINO 9000 - DULCE DE LECHE / FRUTILLA,DOYPACK -160 COMPRIMIDOS,480,12,"$ \n13.566,94","$ \n16.416,00"
7,BCAA 4000,FRASCO - 120 COMPRIMIDOS,168,24,"$ \n8.759,01","$ \n10.598,40"
8,C.D.S. (CREATINE DELIVERY SYSTEM) - FRUTAS TRO...,DOYPACK - POLVO,800,12,"$ \n12.472,07","$ \n15.091,20"
9,CARNITINA,FRASCO - 90 CAPSULAS,41,24,"$ \n7.521,32","$ \n9.100,80"


In [102]:
pdf_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
tables = camelot.read_pdf(pdf_path, flavor='hybrid', pages='1', table_areas=['50,780,800,0'])


df = tables[0].df.copy()

# Limpiar los datos de la tabla, eliminando espacios extra
df = df.map(lambda x: str(x).strip() if isinstance(x, str) else x)

# Eliminar filas completamente vacías
df.dropna(how='all', inplace=True)

# Eliminar la fila de encabezado que a veces se extrae
df = df[~df.iloc[:, 0].str.contains('DESCRIPCIÓN', na=False)]

# Reorganizar los encabezados y datos
df.columns = ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']

df.head(20)

d:\Users\juanp_schamun\AppData\Local\anaconda3\envs\pdf_env\lib\site-packages\camelot\parsers\hybrid.py:161: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table.df = table.df.replace("", np.nan)


,DESCRIPCIÓN,PRESENTACIÓN,CONTENIDO NETO,CANTIDAD POR BULTO,PRECIO UNITARIO NETO,PRECIO UNITARIO CON IVA
0,,,NETO GRAMOS,POR BULTO,UNITARIO NETO,
1,,,,,,IVA
2,,LINEA ALTO RENDIMIENTO,,,,
3,,POTE -150 COMPRIMIDOS,285,12,"$ \n11.424,79","$ \n13.824,00"
4,AMINO 7600,,,,,
5,,POTE - 325 COMPRIMIDOS,618,12,"$ \n22.849,59","$ \n27.648,00"
6,AMINO 9000 - DULCE DE LECHE / FRUTILLA,DOYPACK -160 COMPRIMIDOS,480,12,"$ \n13.566,94","$ \n16.416,00"
7,BCAA 4000,FRASCO - 120 COMPRIMIDOS,168,24,"$ \n8.759,01","$ \n10.598,40"
8,C.D.S. (CREATINE DELIVERY SYSTEM) - FRUTAS TRO...,DOYPACK - POLVO,800,12,"$ \n12.472,07","$ \n15.091,20"
9,CARNITINA,FRASCO - 90 CAPSULAS,41,24,"$ \n7.521,32","$ \n9.100,80"


In [98]:
import re

# Suponiendo que 'df' es el DataFrame resultante de tu código anterior
# df = tables[0].df.copy()
# ... (código de limpieza previo)

# PASO 1: Identificar las filas de categoría y crear una nueva columna
df['Categoria'] = None
current_category = ''

# Iterar sobre las filas y asignar la categoría a los productos subsiguientes
for index, row in df.iterrows():
    # Detectar la fila de categoría usando el texto
    row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
    category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                         'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
    
    is_category_row = any(keyword in row_values_str for keyword in category_keywords)
    
    if is_category_row:
        # Extraer el nombre de la categoría y guardarlo
        category_name = next((val for val in row.tolist() if str(val).strip()), None)
        if category_name:
            # Eliminar la palabra "LINEA" para un nombre más limpio
            current_category = re.sub(r'(LÍNEA|LINEA)\s*', '', str(category_name), flags=re.IGNORECASE).strip()
    else:
        # Asignar la categoría guardada a las filas de productos
        df.at[index, 'Categoria'] = current_category

# PASO 2: Eliminar las filas de categoría originales basadas en la nueva lógica
# La fila se elimina si contiene una palabra clave de categoría y el campo de "DESCRIPCIÓN" está vacío.
rows_to_keep = []
for index, row in df.iterrows():
    # Verificar si el campo "DESCRIPCIÓN" de la fila no está vacío
    is_product_row = str(row['DESCRIPCIÓN']).strip() != ''

    # Si es una fila de producto (con una descripción), la mantenemos
    if is_product_row:
        rows_to_keep.append(True)
    else:
        # Si la descripción está vacía, verificamos si contiene una palabra clave de categoría
        row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
        is_category_row = any(keyword in row_values_str for keyword in category_keywords)

        # Si no es una fila de categoría, la mantenemos, ya que podría ser un sub-producto
        if not is_category_row:
            rows_to_keep.append(True)
        else:
            rows_to_keep.append(False)

df_processed = df[rows_to_keep][2:].copy()

# Opcional: Reordenar las columnas para que 'Categoria' esté al inicio
column_order = ['Categoria'] + [col for col in df_processed.columns if col != 'Categoria']
df_processed = df_processed[column_order]

# Ahora, df_processed es el DataFrame con la columna 'Categoria' y sin las filas de categoría.

df_processed.head(40)


,Categoria,DESCRIPCIÓN,PRESENTACIÓN,CONTENIDO NETO,CANTIDAD POR BULTO,PRECIO UNITARIO NETO,PRECIO UNITARIO CON IVA
3,ALTO RENDIMIENTO,,POTE -150 COMPRIMIDOS,285,12,"$ \n11.424,79","$ \n13.824,00"
4,ALTO RENDIMIENTO,AMINO 7600,,,,,
5,ALTO RENDIMIENTO,,POTE - 325 COMPRIMIDOS,618,12,"$ \n22.849,59","$ \n27.648,00"
6,ALTO RENDIMIENTO,AMINO 9000 - DULCE DE LECHE / FRUTILLA,DOYPACK -160 COMPRIMIDOS,480,12,"$ \n13.566,94","$ \n16.416,00"
7,ALTO RENDIMIENTO,BCAA 4000,FRASCO - 120 COMPRIMIDOS,168,24,"$ \n8.759,01","$ \n10.598,40"
8,ALTO RENDIMIENTO,C.D.S. (CREATINE DELIVERY SYSTEM) - FRUTAS TRO...,DOYPACK - POLVO,800,12,"$ \n12.472,07","$ \n15.091,20"
9,ALTO RENDIMIENTO,CARNITINA,FRASCO - 90 CAPSULAS,41,24,"$ \n7.521,32","$ \n9.100,80"
10,ALTO RENDIMIENTO,CARTÍLAGO DE TIBURÓN,FRASCO - 60 COMPRIMIDOS,72,24,"$ \n6.188,43","$ \n7.488,00"
11,ALTO RENDIMIENTO,CREATINA MASTICABLE - FRUTILLA,POTE -150 COMPRIMIDOS,437,12,"$ \n13.519,34","$ \n16.358,40"
12,ALTO RENDIMIENTO,CREATINA MONOHIDRATO - AFA - KOSHER,DOYPACK - POLVO,250,12,"$ \n16.232,73","$ \n19.641,60"


In [107]:
import pandas as pd
import re
import numpy as np

def procesarDf(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y procesa un DataFrame extraído de un PDF para estructurar los datos
    de productos y categorías.

    Esta función realiza los siguientes pasos:
    1. Limpia los datos eliminando espacios y filas vacías.
    2. Renombra las columnas para una mejor legibilidad.
    3. Propaga las descripciones de productos y las categorías a las filas correspondientes,
       manejando celdas combinadas y saltos de línea.
    4. Elimina las filas de categoría y cualquier otra fila irrelevante.
    5. Elimina las filas de producto que no pudieron ser categorizadas.
    6. Reordena las columnas para una presentación más lógica.

    Args:
        df (pd.DataFrame): El DataFrame original extraído directamente del PDF.

    Returns:
        pd.DataFrame: Un DataFrame limpio y estructurado con productos,
                      sus descripciones y categorías.
    """
    # 1. Limpieza inicial del DataFrame
    # Eliminar espacios extra de todas las celdas
    df = df.applymap(lambda x: str(x).strip() if isinstance(x, str) else x)

    # Eliminar filas que están completamente vacías
    df.dropna(how='all', inplace=True)

    # Eliminar la fila de encabezado que a veces se extrae
    df = df[~df.iloc[:, 0].str.contains('DESCRIPCIÓN', na=False)]

    # 2. Reorganizar los encabezados y datos
    df.columns = ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']

    # 3. Propagación de descripciones de productos y categorías
    df_combined = df.copy()

    # Propagar las descripciones de productos, buscando tanto hacia adelante (ffill)
    # como hacia atrás (bfill) para manejar celdas combinadas.
    df_combined['temp_DESCRIPCIÓN'] = df_combined['DESCRIPCIÓN'].replace('', pd.NA).ffill().bfill()

    # Identificar y propagar las categorías.
    category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                         'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
    
    # Función para encontrar la categoría en una fila.
    def find_category(row):
        row_str = ' '.join(str(val) for val in row.tolist()).upper()
        for keyword in category_keywords:
            if keyword in row_str:
                cleaned_name = re.sub(r'(LÍNEA|LINEA)\s*', '', keyword, flags=re.IGNORECASE).strip()
                return cleaned_name
        return None

    df_combined['Categoria'] = df_combined.apply(find_category, axis=1)
    df_combined['Categoria'] = df_combined['Categoria'].replace('', pd.NA).ffill()

    # 4. Crear el DataFrame final y limpiar
    # Se filtra el DataFrame para incluir solo las filas con un precio válido.
    df_processed = df_combined[df_combined['PRECIO UNITARIO CON IVA'].apply(lambda x: str(x).strip() != '')].copy()

    # Asignar la descripción propagada a la columna principal.
    df_processed['DESCRIPCIÓN'] = df_processed['temp_DESCRIPCIÓN']

    # 5. Limpiar y reordenar el DataFrame final
    df_processed.drop(columns=['temp_DESCRIPCIÓN'], inplace=True)
    df_processed.reset_index(drop=True, inplace=True)

    # Eliminar las filas donde no se pudo asignar una categoría.
    df_processed = df_processed.dropna(subset=['Categoria'])
    df_processed.reset_index(drop=True, inplace=True)

    # Opcional: Reordenar las columnas.
    column_order = ['Categoria', 'DESCRIPCIÓN'] + [col for col in df_processed.columns if col not in ['Categoria', 'DESCRIPCIÓN']]
    df_processed = df_processed[column_order]

    return df_processed





In [108]:
pdf_path = 'LISTA DE PRECIOS MAYORISTA - GENTECH - SEPTIEMBRE 2025.pdf'
tables = camelot.read_pdf(pdf_path, flavor='hybrid', pages='1', table_areas=['50,780,800,0'])


df = tables[0].df.copy()

procesarDf(df)

d:\Users\juanp_schamun\AppData\Local\anaconda3\envs\pdf_env\lib\site-packages\camelot\parsers\hybrid.py:161: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table.df = table.df.replace("", np.nan)
D:\Users\juanp_schamun\AppData\Local\Temp\ipykernel_76340\1120024984.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip() if isinstance(x, str) else x)


,Categoria,DESCRIPCIÓN,PRESENTACIÓN,CONTENIDO NETO,CANTIDAD POR BULTO,PRECIO UNITARIO NETO,PRECIO UNITARIO CON IVA
0,ALTO RENDIMIENTO,AMINO 7600,POTE -150 COMPRIMIDOS,285,12,"$ \n11.424,79","$ \n13.824,00"
1,ALTO RENDIMIENTO,AMINO 7600,POTE - 325 COMPRIMIDOS,618,12,"$ \n22.849,59","$ \n27.648,00"
2,ALTO RENDIMIENTO,AMINO 9000 - DULCE DE LECHE / FRUTILLA,DOYPACK -160 COMPRIMIDOS,480,12,"$ \n13.566,94","$ \n16.416,00"
3,ALTO RENDIMIENTO,BCAA 4000,FRASCO - 120 COMPRIMIDOS,168,24,"$ \n8.759,01","$ \n10.598,40"
4,ALTO RENDIMIENTO,C.D.S. (CREATINE DELIVERY SYSTEM) - FRUTAS TRO...,DOYPACK - POLVO,800,12,"$ \n12.472,07","$ \n15.091,20"
5,ALTO RENDIMIENTO,CARNITINA,FRASCO - 90 CAPSULAS,41,24,"$ \n7.521,32","$ \n9.100,80"
6,ALTO RENDIMIENTO,CARTÍLAGO DE TIBURÓN,FRASCO - 60 COMPRIMIDOS,72,24,"$ \n6.188,43","$ \n7.488,00"
7,ALTO RENDIMIENTO,CREATINA MASTICABLE - FRUTILLA,POTE -150 COMPRIMIDOS,437,12,"$ \n13.519,34","$ \n16.358,40"
8,ALTO RENDIMIENTO,CREATINA MONOHIDRATO - AFA - KOSHER,DOYPACK - POLVO,250,12,"$ \n16.232,73","$ \n19.641,60"
9,ALTO RENDIMIENTO,CREATINA MONOHIDRATO - AFA - KOSHER,DOYPACK - POLVO,500,12,"$ \n29.038,02","$ \n35.136,00"


In [ ]:

products = []
current_category = 'Sin Categoria'
last_product_desc = ""

for _, row in df.iterrows():
    row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
    
    # Detectar las filas de categorías
    category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                            'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
    is_category_row = any(keyword in row_values_str for keyword in category_keywords)

    if is_category_row:
        category_name = next((val for val in row.tolist() if str(val).strip()), None)
        if category_name:
            cleaned_name = re.sub(r'(LÍNEA|LINEA)\s*', '', str(category_name), flags=re.IGNORECASE).strip()
            if cleaned_name.upper() not in ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']:
                current_category = cleaned_name
        continue

# Si la fila tiene una descripción no vacía, es un producto principal
if str(row['DESCRIPCIÓN']).strip():
    last_product_desc = str(row['DESCRIPCIÓN'])

for _, row in df.iterrows():
    row_values_str = ' '.join(str(val) for val in row.tolist()).upper()
    
    # Detectar las filas de categorías
    category_keywords = ['LINEA ALTO RENDIMIENTO', 'LINEA PREMIUM', 'LINEA IRON', 'LINEA BEAUTY', 'LINEA PRE_WORKOUT', 'LINEA NUTRICION',
                            'LINEA KIDS', 'LINEA VEGGIE PLANT BASED', 'ACCESORIOS']
    is_category_row = any(keyword in row_values_str for keyword in category_keywords)

    if is_category_row:
        category_name = next((val for val in row.tolist() if str(val).strip()), None)
        if category_name:
            cleaned_name = re.sub(r'(LÍNEA|LINEA)\s*', '', str(category_name), flags=re.IGNORECASE).strip()
            if cleaned_name.upper() not in ['DESCRIPCIÓN', 'PRESENTACIÓN', 'CONTENIDO NETO', 'CANTIDAD POR BULTO', 'PRECIO UNITARIO NETO', 'PRECIO UNITARIO CON IVA']:
                current_category = cleaned_name
        continue

   


df.head(20)
